In [1]:
import tensorflow as tf
import os
import time
import numpy as np
import pandas as pd
import PIL.Image as Image
from tensorflow.keras.applications.resnet50 import preprocess_input

Load image -> batching into 4D-Tensor -> pre-processing

In [2]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [3]:
df = pd.read_csv("./CV_data/fix_split_test.csv")
sub_df = pd.DataFrame(df.iloc[:, 5:10].values, 
                      columns=['xmin', 'ymin', 'xmax', 'ymax','id'])

In [4]:
sub_df=sub_df.sort_values(by=['id'])
sub_df = sub_df.reset_index(drop=True)
sub_df = sub_df.astype('int')

In [5]:
my_id_1 = 390369
path_1 = './CV_data/'+str(my_id_1)+'.jpg'
image_1 = load_image_into_numpy_array(Image.open(path_1))
wdf_1= sub_df[sub_df['id']==my_id_1].copy()

In [ ]:
my_id_2 = 4801764348
path_2 = './CV_data/'+str(my_id_2)+'.jpg'
image_2 = load_image_into_numpy_array(Image.open(path_2))
wdf_2= sub_df[sub_df['id']==my_id_2].copy()

In [6]:
batch_size =len(wdf_1)+len(wdf_2)
hw = 224
# hw = 331
batch = np.zeros((batch_size,hw,hw,3))
for i,row in wdf.iterrows():
    img = image[row.ymin:row.ymax+1,row.xmin:row.xmax+1,:]
    img = np.resize(img,(hw,hw,3))
    batch[i] = img

batch = preprocess_input(batch)

Load frozen model

In [7]:
DIR_NAME = './TF_models/'
MODEL_NAME = 'resnet50'
PATH_TO_FROZEN_GRAPH = DIR_NAME + MODEL_NAME + '.pb'

In [8]:
# Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [9]:
def get_feature(batch_image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            tensor_dict = {}
#             key= 'avg_pool/Mean'
#             key= 'fc2/Relu'
#             key= 'global_average_pooling2d_2/Mean'
            tensor_name = key + ':0'
            
            tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
            
            image_tensor = tf.get_default_graph().get_tensor_by_name('input_4:0')
            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: batch_image})
    return output_dict

In [10]:
start = time.clock()
output_dict = get_feature(batch, detection_graph)
end=time.clock()
print(end-start)
# print(output_dict['avg_pool/Mean'].shape)
print(output_dict['fc2/Relu'].shape)
# print(output_dict['global_average_pooling2d_2/Mean'].shape)

13.006689
(8, 4096)
